# Environment

In [1]:
run_component_tests = False

In [2]:
# import time
# from datetime import datetime, timedelta
# import traceback
# import os
# import pandas as pd
# import numpy as np
# import subprocess as sp
# import multiprocessing as mp
# import json

#system = 'Windows'
system = 'Linux'
#system = 'Raspberry'

system_dict = {
    'Windows':{
        'rootdir':r'C:\Users\Christoph\Documents\SmartInverter\smartinverter_controller\RaspberryPi\TestService\ram',
        'storedir':r'C:\Users\Christoph\Documents\SmartInverter\smartinverter_controller\RaspberryPi\TestService\sd',
        'optimization_root':r'C:\Users\Christoph\Documents\SmartInverter\smartinverter_optimization',
    },
    'Linux':{
        'rootdir':'/home/christoph/Documents/SmartInverter/smartinverter_controller/RaspberryPi/TestService/ram',
        'storedir':'/home/christoph/Documents/SmartInverter/smartinverter_controller/RaspberryPi/TestService/sd',
        'optimization_root':'/home/christoph/Documents/SmartInverter/smartinverter_optimization',
    },
    'Raspberry':{
        'rootdir':'/var/sitmp',
        'storedir':'/home/ubunut',
        'optimization_root':'',
    },
}
    

rootdir = system_dict[system]['rootdir'] # Directory on RAM
storedir = system_dict[system]['storedir'] # Directory on SD-card
optimization_root = system_dict[system]['optimization_root']

import os
import sys
sys.path.append(optimization_root)
sys.path.append(os.path.join(optimization_root, 'SlowActing'))
sys.path.append(os.path.join(optimization_root, 'Forecasting'))

if run_component_tests:
    import matplotlib.pyplot as plt
    %matplotlib inline

# Triggering Class

In [3]:
import time

class triggering(object):
    def __init__(self, ts):
        self.ts = ts
        self.initialize_all_trigger() 

    def initialize_all_trigger(self):
        now = time.time()
        self.trigger = {}
        for k in self.ts.keys():
            self.trigger[k] = self.get_trigger(now, self.ts[k], mode='prev', integer=self.ts[k]>1)

    def refresh_trigger(self, k, now):
        self.trigger[k] = self.get_trigger(now, self.ts[k], mode='next', integer=self.ts[k]>1)

    def get_trigger(self, t, ts, mode='next', integer=True):
        trigger = round(t/ts)*ts
        if integer: trigger = int(trigger)
        if mode == 'next': trigger = trigger + ts
        elif mode == 'prev': trigger = trigger + ts
        return trigger

In [4]:
if run_component_tests:
    ts = {} 
    ts['main'] = 0.5 # seconds
    ts['print'] = 1
    
    trigger_test = triggering(ts)
    now_init = time.time()
    now = now_init
    while now < now_init+2:
        now = time.time()
        if now >= trigger_test.trigger['main']: print 'Main triggered', now; trigger_test.refresh_trigger('main', now)
        if now >= trigger_test.trigger['print']: print 'Print triggered', now; trigger_test.refresh_trigger('print', now)

Main triggered 1541023930.5
Main triggered 1541023931.0
Print triggered 1541023931.0
Main triggered 1541023931.5
Main triggered 1541023932.0
Print triggered 1541023932.0


# Controller Base Class (eFMU)

In [5]:
import abc
import warnings

class eFMU(object):
    __metaclass__  = abc.ABCMeta
    
    def __init__(self):
        self.input = {}     
        self.output = {}      
    def do_step(self, inputs={}):
        self.set_inputs(inputs)
        return self.compute()
    def get_model_variables(self):
        return self.input.keys()
    def set_real(self, name, value):
        self.input[name] = value
    def set_inputs(self, inputs):
        all_keys = self.input.keys()
        for k, v in inputs.iteritems():
            if k in all_keys:
                self.set_real(k, v)
                all_keys.remove(k)
            else:
                raise KeyError('{} not in input list.'.format(k))
        if len(all_keys) > 0:
            warnings.warn('Not all input specified, but continue step. Missing keys:{}'.format(all_keys), Warning)
    @abc.abstractmethod
    def compute(self):
        ''' Mathod to compute the output '''
    def get_output(self, keys=[]):
        if keys == []:
            return self.output
        else:
            out = {}
            for k in keys:
                if k in self.output.keys():
                    out[k] = self.output[k]
                else:
                    raise KeyError('{} not in output list.'.format(k))
            return out
    def get_input(self, keys=[]):
        if keys == []:
            return self.input
        else:
            out = {}
            for k in keys:
                if k in self.input.keys():
                    out[k] = self.input[k]
                else:
                    raise KeyError('{} not in input list.'.format(k))
            return out

In [6]:
if run_component_tests:
    class testcontroller1(eFMU):
        def __init__(self):
            self.input = {'a':None,'b':None}
            self.output = {'c':None}
        def compute(self):
            self.output['c'] = self.input['a'] * self.input['b']
    # Test controller
    testcontroller = testcontroller1()
    # Get all variables
    variables = testcontroller.get_model_variables()
    # Makeup some inputs
    inputs = {}
    for var in variables:
        inputs[var] = 2
    # Query controller
    print 'Log-message', testcontroller.do_step(inputs=inputs) # Display log message
    print 'Input', testcontroller.input
    print 'Output', testcontroller.output

Log-message None
Input {'a': 2, 'b': 2}
Output {'c': 4}


# Controller Stack Class

In [10]:
import pandas as pd
import numpy as np
from copy import deepcopy as copy_dict

from RunEmulationEnvironment import PythonDB_wrapper
from Pandas_db_API import status_db, write_db, read_db

class controller_stack(object):
    def __init__(self, controller, tz=-8, debug=False, name='Zone1'):
        self.controller = controller
        self.tz = tz
        self.debug = debug
        self.name = name

    def initialize(self, mapping, now=time.time()):
        self.db_mode = 'Python_db'
        self.initialize_controller(now)
        self.initialize_database()
        self.initialize_mapping(mapping)
        
    def initialize_controller(self, now):
        for name, ctrl in self.controller.iteritems():
            ctrl['fun'] = ctrl['fun']()
            ctrl['last'] = 0
            ctrl['inputs'] = ctrl['fun'].input.keys()
            ctrl['outputs'] = ctrl['fun'].output.keys()
            ctrl['log'] = {}
            ctrl['input'] = {}
            ctrl['output'] = {}
            ctrl['log'][now] = ['Initialize']
            ctrl['input'][now] = {}
            ctrl['output'][now] = {}
            
    def initialize_database(self):
        self.database = PythonDB_wrapper(self.name, self.db_mode)
        self.database.address = '127.0.0.1:'+str(self.database.port)
        db_columns = {}
        for name, ctrl in self.controller.iteritems():
            for i in ctrl['inputs']:
                db_columns[name+'_'+i] = -1
            for o in ctrl['outputs']:
                db_columns[name+'_'+o] = -1
        db_columns['timezone'] = self.tz
        db_columns['dev_debug'] = self.debug
        db_columns['dev_nodename'] = self.name
        write_db(db_columns, self.database.address)
        if self.debug: print 'SetupDB\n', read_db(self.database.address)
            
    def initialize_mapping(self, mapping):
        m = mapping.keys()
        for name, ctrl in self.controller.iteritems():
            for c in ctrl['inputs']:
                if name+'_'+c in m: m.remove(name+'_'+c)
                else: raise KeyError('{} not in mapping.'.format(name+'_'+c))
        if len(m) <> 0:
            raise KeyError('{} not a control parameter.'.format(m))
        self.mapping = mapping
            
    def refresh_device_from_db(self):
        self.tz = int(self.data_db['timezone'])
        self.debug = bool(self.data_db['dev_debug'])
        self.name = str(self.data_db['dev_nodename'])

    def query_control(self, now, return_db=False):
        #time_st = time.time()
        #self.data_db = read_db(self.database.address)
        #self.refresh_device_from_db()
        #if self.debug: print 'ReadDB\n', self.data_db
        all_controller = self.controller.keys()
        executed_controller = []
        while len(all_controller) > 0:
            name = all_controller[0]
            ctrl = self.controller[name]
            if type(ctrl['sampletime']) == type(''):
                if ctrl['sampletime'] not in all_controller:
                    if ctrl['sampletime'] in executed_controller:
                        self.do_control(name, ctrl, now)
                        executed_controller.append(name)
                    all_controller.remove(name)
                else:
                    all_controller.remove(name)
                    all_controller.append(name)
            else:
                if now >= (ctrl['last'] + ctrl['sampletime']):
                    self.do_control(name, ctrl, now)
                    executed_controller.append(name)
                all_controller.remove(name)
        if executed_controller <> [] and return_db:
            return self.data_db
        #if self.debug: print 'Duration query_control:',time.time()-time_st
            
    def do_control(self, name, ctrl, now):
        # Query controller
        #print name
        #print read_db(self.database.address)
        #time_st = time.time()
        if self.debug: print 'QueryCTRL', name, 'at', now
        # Refresh inputs
        self.update_inputs(name, ctrl, now)
        #time_st_step = time.time()
        ctrl['log'][now] = ctrl['fun'].do_step(inputs=ctrl['input'][now])
        #if self.debug: print 'Duration do_step for {}: {}'.format(name, time.time()-time_st_step)
        ctrl['output'][now] = copy_dict(ctrl['fun'].output)
        self.log_to_db(name, ctrl, now)
        #ctrl['last'] = time.time()
        ctrl['last'] = now
        #if self.debug: print 'Duration do_control for {}: {}'.format(name, time.time()-time_st)
        
                
    def update_inputs(self, name, ctrl, now):
        self.read_from_db()
        #self.data_db = read_db(self.database.address)
        #self.refresh_device_from_db()
        #if self.debug: print 'ReadDB\n', self.data_db
        inputs = {}
        for c in ctrl['inputs']:
            mapping = self.mapping[name+'_'+c]
            if type(mapping) == type(0) or type(mapping) == type(0.0):
                inputs[c] = mapping
            elif type(mapping) == type([]):
                raise KeyError('Not implemented '+str(mapping))
            else:
                if self.mapping[name+'_'+c] in self.data_db.keys():
                    inputs[c] = self.data_db[self.mapping[name+'_'+c]]
                else:
                    inputs[c] = self.mapping[name+'_'+c]
        ctrl['input'][now] = inputs
        
    def read_from_db(self, name=None, refresh_device=True):
        #time_st = time.time()
        self.data_db = read_db(self.database.address)
        if refresh_device: self.refresh_device_from_db()
        #if self.debug: print 'Duration read_from_db for {}: {}'.format(name, time.time()-time_st)
        
        
    def log_to_db(self, name, ctrl, now):
        #time_st = time.time()
        temp = {}
        for k, v in ctrl['input'][now].iteritems():
            temp[name+'_'+k] = v
        for k, v in ctrl['output'][now].iteritems():
            temp[name+'_'+k] = v
        write_db(temp, self.database.address)
        self.data_db.update(temp)
        #if self.debug: print 'Duration log_to_db for {}: {}'.format(name, time.time()-time_st)

    def log_to_df(self, stacks='all', which=['input','output','log']):
        if stacks == 'all':
            controller = self.controller
        else:
            raise ValueError('Only stacks == "all" implemented!')
        dfs = {}
        for name, ctrl in controller.iteritems():
            if len(ctrl['log']) > 0:
                init = True
                for w in which:
                    if w == 'log':
                        if hasattr(ctrl['fun'], 'columns'): index = ctrl['fun'].columns
                        else: index = ['Logging']
                        temp = pd.DataFrame(ctrl[w], index=index).transpose()
                    else:
                        temp = pd.DataFrame(ctrl[w]).transpose()
                    temp.index = pd.to_datetime(temp.index, unit='s', utc=True).tz_localize(None) + pd.DateOffset(hours=self.tz)
                        
                    if init:
                        dfs[name] = temp.copy(deep=True)
                        init = False
                    else:
                        dfs[name] = pd.concat([dfs[name], temp], axis=1)
        return dfs

    def log_to_csv(self, new=False, path='log'):
        dfs = self.log_to_df()
        mode = 'ab' if not new else 'wb'
        for name, log in dfs.iteritems():
            log.to_csv(path+'_'+name+'.csv')

    def clear_logs(self):
        for name, ctrl in self.controller.iteritems():
            for t in ['output','input','log']:
                ctrl[t] = {}
            
    def shutdown(self):
        self.database.kill_db()
        
    def set_input(self, inputs):
        for k, v in inputs.iteritems():
            if k in self.mapping.keys():
                self.mapping[k] = v
            else:
                raise KeyError('{} not a control parameter.'.format(k))
                
    def get_output(self, name, keys=[]):
        return self.controller[name]['fun'].get_output(keys=keys)
    def get_input(self, name, keys=[]):
        return self.controller[name]['fun'].get_input(keys=keys)
        
# FIXME
# Check for error when db communication
# Parallelize

In [14]:
if run_component_tests:
    controller = {}
    controller['forecast1'] = {'fun':testcontroller1, 'sampletime':1}
    controller['mpc1'] = {'fun':testcontroller1, 'sampletime':'forecast1'}
    # Initialize controller
    controller = controller_stack(controller, tz=-8, debug=True)
    mapping = {}
    mapping['forecast1_a'] = 10
    mapping['forecast1_b'] = 4
    mapping['mpc1_a'] = 'forecast1_c'
    mapping['mpc1_b'] = 'forecast1_a'
    controller.initialize(mapping)
    for i in range(6):
        #print i
        if i == 0: write_db({'dev_debug':False}, controller.database.address); print 'Debug=False'
        #if i == 2: controller.set_input({'ctrl1_b':10})
        #if i == 4: print controller.log_to_df(which=['input'])['ctrl1']; controller.clear_logs(); print 'Clear Logs'
        controller.query_control(time.time())
        print i, controller.data_db['mpc1_c']
        time.sleep(1)

    controller.shutdown()
    
    print '\n\nInput\n', controller.log_to_df(which=['input'])['mpc1']
    print '\n\nLog\n', controller.log_to_df()

SetupDB
{u'forecast1_a': -1, u'dev_time': u'2018-10-31 15:15:41', u'mpc1_a': -1, u'mpc1_b': -1, u'mpc1_c': -1, u'dev_nodename': u'Zone1', u'dev_debug': True, u'forecast1_b': -1, u'forecast1_c': -1, u'dev_error': u'NA', u'dev_status': u'ok', u'dev_port': u'25379', u'timezone': -8}
Debug=False
QueryCTRL forecast1 at 1541024141.26
0 400
1 400
2 400
3 400
4 400
5 400


Input
                                  a     b
2018-10-31 14:14:13.374232054   NaN   NaN
2018-10-31 14:15:41.261368990  40.0  10.0
2018-10-31 14:15:42.267426968  40.0  10.0
2018-10-31 14:15:43.276921034  40.0  10.0
2018-10-31 14:15:44.288898945  40.0  10.0
2018-10-31 14:15:45.299104929  40.0  10.0
2018-10-31 14:15:46.310218096  40.0  10.0


Log
{'forecast1':                                   a    b     c     Logging
2018-10-31 14:14:13.374232054   NaN  NaN   NaN  Initialize
2018-10-31 14:15:41.261368990  10.0  4.0  40.0        None
2018-10-31 14:15:42.267426968  10.0  4.0  40.0        None
2018-10-31 14:15:43.276921034  10.